In [ ]:
from dotenv import load_dotenv
import os

load_dotenv(override=True)
REPO_DIR = os.getenv("DINOV3_DIR")
DINOV3_WEIGHTS = os.getenv("DINOV3_WEIGHTS")
DINOV3_BACKBONE = os.getenv("DINOV3_BACKBONE")


In [ ]:
print(REPO_DIR)
print(DINOV3_WEIGHTS)

In [ ]:
from PIL import Image
import torch
from torchvision import transforms
import matplotlib.pyplot as plt
from torchvision.transforms import v2
from matplotlib import colormaps
from functools import partial
from dinov3.eval.segmentation.inference import make_inference

In [ ]:
print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("CUDA device count:", torch.cuda.device_count())
if torch.cuda.is_available():
    print("GPU device name:", torch.cuda.get_device_name(0))

In [ ]:
def get_img():
    import requests
    url = "http://images.cocodataset.org/val2017/000000039769.jpg"
    image = Image.open(requests.get(url, stream=True).raw).convert("RGB")
    return image

In [ ]:
def make_transform(resize_size: int | list[int] = 768):
    to_tensor = v2.ToImage()
    resize = v2.Resize((resize_size, resize_size), antialias=True)
    to_float = v2.ToDtype(torch.float32, scale=True)
    normalize = v2.Normalize(
        mean=(0.485, 0.456, 0.406),
        std=(0.229, 0.224, 0.225),
    )
    return v2.Compose([to_tensor, resize, to_float, normalize])

In [ ]:
segmentor = torch.hub.load(REPO_DIR, 'dinov3_vit7b16_ms', source="local", weights=DINOV3_WEIGHTS,backbone_weights=DINOV3_BACKBONE)

In [ ]:
img_size = 896
img  = get_img()
transform = make_transform(img_size)
with torch.inference_mode():
    with torch.autocast('cuda', dtype=torch.bfloat16):
        batch_img = transform(img)[None]
        pred_vit7b = segmentor(batch_img)  # raw predictions  
        # actual segmentation map
        segmentation_map_vit7b = make_inference(
            batch_img,
            segmentor,
            inference_mode="slide",
            decoder_head_type="m2f",
            rescale_to=(img.size[-1], img.size[-2]),
            n_output_channels=150,
            crop_size=(img_size, img_size),
            stride=(img_size, img_size),
            output_activation=partial(torch.nn.functional.softmax, dim=1),
        ).argmax(dim=1, keepdim=True)
plt.figure(figsize=(12, 6))
plt.subplot(121)
plt.imshow(img)
plt.axis("off")
plt.subplot(122)
plt.imshow(segmentation_map_vit7b[0,0].cpu(), cmap=colormaps["Spectral"])
plt.axis("off")